In [5]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Cargar el dataset de Diabetes
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df["DiseaseProgression"] = diabetes.target  # Variable objetivo

# Seleccionar variables independientes (X) y la variable objetivo (y)
X = df.drop(columns=["DiseaseProgression"])  # Variables predictoras
y = df["DiseaseProgression"]  # Variable objetivo

# Comprobamos el rango de valores de la variable objetivo
print(f"Valor mínimo de DiseaseProgression: {y.min()}")
print(f"Valor máximo de DiseaseProgression: {y.max()}")

# Escalar las variables independientes con StandardScaler()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Devuelve un array NumPy

# Calcular la correlación con la variable objetivo
df_scaled = pd.DataFrame(X_scaled, columns=X.columns) 
df_scaled["DiseaseProgression"] = y  # Agregamos la variable objetivo para calcular correlación
correlaciones = df_scaled.corr()["DiseaseProgression"].sort_values(ascending=False)

# Calcular el VIF para detectar colinealidad entre variables
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Dividir el dataset en entrenamiento (80%) y prueba (20%) con train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Regresión Lineal con LinearRegression()
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Obtener e interpretar los coeficientes y el intercepto del modelo
coeficientes = pd.Series(modelo.coef_, index=X.columns)

print("Intercepto:", modelo.intercept_)
print("Coeficientes del modelo:")
print(coeficientes)

# Realizar predicciones con modelo.predict(X_test)
y_pred = modelo.predict(X_test)

# Evaluar el modelo 
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print(f"MAE (Error Absoluto Medio): {mae:.4f}")
print(f"MSE (Error Cuadrático Medio): {mse:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R² (Coeficiente de Determinación): {r2:.4f}")

Valor mínimo de DiseaseProgression: 25.0
Valor máximo de DiseaseProgression: 346.0
Intercepto: 151.3456045398599
Coeficientes del modelo:
age     1.802910
sex   -11.509071
bmi    25.800705
bp     16.538585
s1    -44.306406
s2     24.641710
s3      7.773096
s4     13.095537
s5     35.017409
s6      2.315027
dtype: float64
MAE (Error Absoluto Medio): 42.7941
MSE (Error Cuadrático Medio): 2900.1936
RMSE (Raíz del Error Cuadrático Medio): 53.8534
R² (Coeficiente de Determinación): 0.4526


## **Análisis e interpretación del modelo de regresión lineal múltiple - Diabetes Dataset**

Este modelo de regresión lineal múltiple intenta predecir la **progresión de la diabetes** en pacientes basándose en diversas variables médicas. Sin embargo, los resultados indican que el modelo **no tiene una gran precisión**. Podemos concluir esto en base a los resultados que nos arroja el modelo, que vamos a ir analizando uno por uno:

---

### **MAE (Error Absoluto Medio)**
**Valor obtenido:** `42.7941`  

El **Error Absoluto Medio (MAE)** indica el error promedio entre las predicciones del modelo y los valores reales. En este caso, el modelo tiene un **error promedio de aproximadamente 43 unidades**, lo que significa que las predicciones pueden desviarse considerablemente de los valores reales. Aunque el MAE da una idea del error medio, no diferencia entre errores pequeños y grandes, por lo que analizaremos también otros indicadores.  

---

### **MSE (Error Cuadrático Medio)**
**Valor obtenido:** `2900.1936`  

El **Error Cuadrático Medio (MSE)** mide el promedio de los errores elevados al cuadrado. Penaliza más los errores grandes, lo que lo hace útil para detectar **predicciones extremadamente alejadas de los valores reales**. Un **MSE alto** indica que el modelo **comete errores significativos**, especialmente errores grandes. Aunque este valor es útil, su magnitud es difícil de interpretar, por lo que usamos su raíz cuadrada (**RMSE**) para tener una mejor idea del error medio en las unidades originales.  

---

### **RMSE (Raíz del Error Cuadrático Medio)**
**Valor obtenido:** `53.8534`  

El **RMSE** es la raíz cuadrada del MSE . En este caso, un **RMSE de aproximadamente 54 unidades** significa que el modelo, en promedio, comete errores de **más de 50 unidades** al predecir la progresión de la diabetes. El **RMSE alto** confirma que el modelo **no es muy preciso** en sus predicciones. 

---

### **R² (Coeficiente de Determinación)**
**Valor obtenido:** `0.4526`  

Finalmente, el **coeficiente R²** indica **qué porcentaje de la variabilidad en la progresión de la diabetes es explicada por el modelo**. Un **R² de 0.4526** significa que **solo el 45.26% de la variabilidad** en la progresión de la diabetes puede ser explicada por las variables incluidas en el modelo.  Esto **indica que hay otros factores importantes** (no considerados en el modelo) que afectan la progresión de la enfermedad. 
 
**Conclusión:** El modelo **explica menos de la mitad de la variabilidad**, lo que confirma que **no es un buen modelo predictivo** para esta tarea. Este modelo nos da una idea general sobre qué factores pueden influir en la progresión de la diabetes, pero **no es suficientemente preciso** para hacer predicciones confiables.


In [9]:
from sklearn.preprocessing import MinMaxScaler

# Eliminar las variables sex y s1
X_modified = df.drop(columns=["DiseaseProgression", "sex", "s1"])

# Aplicar MinMaxScaler en lugar de StandardScaler
scaler = MinMaxScaler()
X_scaled_modified = scaler.fit_transform(X_modified)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled_modified, y, test_size=0.2, random_state=42)

# Entrenar el modelo con las nuevas variables
modelo_modificado = LinearRegression()
modelo_modificado.fit(X_train, y_train)

# Obtener predicciones
y_pred_modificado = modelo_modificado.predict(X_test)

# Evaluar el nuevo modelo
mae_mod = mean_absolute_error(y_test, y_pred_modificado)
mse_mod = mean_squared_error(y_test, y_pred_modificado)
rmse_mod = np.sqrt(mse_mod)
r2_mod = r2_score(y_test, y_pred_modificado)

# Mostrar resultados comparativos
print(f"Nuevo MAE: {mae_mod:.4f} (Original: {mae:.4f})")
print(f"Nuevo MSE: {mse_mod:.4f} (Original: {mse:.4f})")
print(f"Nuevo RMSE: {rmse_mod:.4f} (Original: {rmse:.4f})")
print(f"Nuevo R²: {r2_mod:.4f} (Original: {r2:.4f})")


Nuevo MAE: 44.0293 (Original: 42.7941)
Nuevo MSE: 2952.4276 (Original: 2900.1936)
Nuevo RMSE: 54.3362 (Original: 53.8534)
Nuevo R²: 0.4427 (Original: 0.4526)


### **Conclusión del análisis tras los ajustes en el modelo**  

Después de eliminar las variables `sex` y `s1` y cambiar la normalización de StandardScaler a MinMaxScaler, los resultados han empeorado ligeramente en lugar de mejorar. El R² ha disminuido de 0.4526 a 0.4427, lo que indica que el modelo ahora explica aún menos variabilidad en la progresión de la diabetes. Además, los errores han aumentado (MAE de 42.79 a 44.03 y RMSE de 53.85 a 54.33), lo que sugiere que estos ajustes no han contribuido a mejorar la precisión del modelo.

Estos resultados refuerzan la idea de que la regresión lineal múltiple no es el modelo adecuado para este problema. La progresión de la diabetes probablemente no sigue una relación lineal con las variables del dataset, lo que limita el desempeño del modelo. Para mejorar las predicciones, sería más adecuado explorar modelos más avanzados como Random Forest o Redes Neuronales, que pueden capturar relaciones más complejas entre las variables.  

